# Functional Programming Skills:  Higher Order Functions
Functions in Python are ["first class citizens"](https://realpython.com/lessons/functions-first-class-objects-python/)
That means a function
 - is just an object, with a type, and can be treated like any other object
 - can be passed as a parameter to other functions
 - can be returned as a function's value

A ["higher order function"](https://python.plainenglish.io/higher-order-functions-in-python-for-beginners-8155e67ea013) is a function that takes a function as an input or returns a function as its output.

That sounds strange, but once you get the hang of it, it is an increadibly powerful way to re-use code.

In [107]:
import math, time

### Example:  Simple Closure
A function that returns another function creates a ["closure"](https://en.wikipedia.org/wiki/Closure_(computer_programming). This just means that the inner function returned retains the value any parameteres or local variables defined by the outer function...

In [108]:
def power(exponent):
    """ A factory that returns the power function for the given exponent """
    def f(n):
        return n**exponent
    return f

square = power(2)
cube = power(3)

assert square(5) == 25
assert cube(3) == 27

print(f'square is a {type(square)}.  square(9)=={square(9)}')

square is a <class 'function'>.  square(9)==81


### Example: functional arguments
The built-in python functions that perform ordering operations on a sequence, like `min`, `max`, `sorted`, take an optional argument (usually named `key`) that is a function used to define the value to order on...

In [109]:
def sort(lst, item=lambda item: item):
    """ Sort the list in place with the order determined by the given function """
    # Bubble sort - one of the first sort algorithm most CS students learn.  Pretty inefficient.
    for i in range(len(lst)):
        for j in range(1, len(lst)-i):
            if not item(lst[j-1]) < item(lst[j]):
                lst[j-1], lst[j] = lst[j], lst[j-1]
    return lst

print(f'Numeric sort: {sort([6, 3, 9, 2, 5, 7, 1, 0, 8, 4])}')

people = [
    dict(name='bob', gender='male', weight=70, height=180, age=45),
    dict(name='bei', gender='female', weight=55, height=175, age=35),
    dict(name='kai', gender='male', weight=90, height=210, age=24),
    dict(name='kia', gender='female', weight=60, height=185, age=27),
]
print('Sort records by age...')
sort(people, item=lambda person: person['age'])

Numeric sort: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Sort records by age...


[{'name': 'kai', 'gender': 'male', 'weight': 90, 'height': 210, 'age': 24},
 {'name': 'kia', 'gender': 'female', 'weight': 60, 'height': 185, 'age': 27},
 {'name': 'bei', 'gender': 'female', 'weight': 55, 'height': 175, 'age': 35},
 {'name': 'bob', 'gender': 'male', 'weight': 70, 'height': 180, 'age': 45}]

### Map, Filter, Reduce
We've seen [`map`](https://docs.python.org/3/library/functions.html#map) and [`filter`](https://docs.python.org/3/library/functions.html#filter) are very common operations.  The third operation in this set is [`reduce`](https://docs.python.org/3/library/functools.html?highlight=functools#functools.reduce), which combines all the elements from an iterable into a single value (e.g., [`sum`](https://docs.python.org/3/library/functions.html#sum) is a reduce operation)
Python provides built-ins for each these operations.  Each takes an iterable and function as input and returns a generator.

Just for giggles, let's look at what `reduce` does...

In [110]:
def reduce(iterable, combine, initial=0):
    """ Combine all items in the iterable by accumulating them in a running total initialized to initial. """
    total = initial
    for item in iterable:
        total = combine(total, item)      # where we successively accumulate items into the running total
    return total

# Python's built-in sum function is a special case of reduce:
assert sum(range(10)) == reduce(range(10), lambda x, y: x+y)

print(f'Factorial of 10 is {reduce(range(1,11), lambda x, y: x*y, initial=1)}')

Factorial of 10 is 3628800


### Putting it all together:  a reduce factory
The `reduce` function defined above is very generic, but hard to use because it doesn't provide an abstraction for the specific reduction.

Here we create a "closure" to return a specific version of `reduce` for a specific `combine` function.
You could think of the outer function, `reducer` as a "factory" that builds specific reduce operations.

In [111]:
def reducer(combine, initial=0):
    """ Return a reduce function using a specific combine function. """
    def reduce(iterable):
        total = initial
        for item in iterable:
            total = combine(total, item)      # where we successively accumulate items into the running total
        return total
    return reduce

# build a reducer similar to python's built-in sum function:
my_sum = reducer(lambda x, y: x+y)

N = 10
assert sum(range(N)) == my_sum(range(N))

product = reducer(lambda x, y: x*y, initial=1)
print(f'Factorial of {N} is {product(range(1,N+1))}')

Factorial of 10 is 3628800


## Decorators
A **decorator** is a function that "wraps" another function in a closure, but also adds some extra behaviour.
Here's a minimal example...

In [112]:
def double(f):
    """ return a function that doubles the output of function f """
    def doubler(n):
        return 2*f(n)
    return doubler


def factorial(n):
    """ Return factorial of n -- notice this is a reduce operation! """
    f = n
    while n > 1:
        n -= 1
        f *= n
    return f

assert factorial(5) == 120

double_factorial = double(factorial)

assert double_factorial(5) == 240

In the example above, we call function `double` a "decorator"
and say that `double_factorial` is a "decorated" version of `factorial`

## Practical Decorators
We'll develop a performance timing decorator and a caching decorator below,
but first we need a problem that is computationally expensive...

In [113]:
# (Note: there are MUCH more efficient ways to identify prime numbers! These naive algorithms are for illustration only.)

def is_prime(n):
    """ Return True iff integer n is a prime number """
    assert type(n) is int and n >= 2
    div, upper = 2, math.sqrt(n)
    while div <= upper:
        if n%div == 0:
            return False
        div += 1
    return True

assert is_prime(11)

N = 1246739743737
print(f"{N} is {'' if is_prime(N) else 'not'} prime.")


def find_next_prime(n):
    """ Return the first prime number at least as large as n """
    assert type(n) is int and n >= 1
    while not is_prime(n):
        n += 1
    return n

assert find_next_prime(23+1) == 29

def nth_prime(n):
    """ Return the n'th prime number """
    assert type(n) is int and n >= 1
    nth = 1
    prime = 2
    while n > nth:
        prime = find_next_prime(prime+1)
        nth += 1
    return prime

assert nth_prime(10) == 29

Nth = 20000
print(f"The {Nth}'th prime number is {nth_prime(Nth)}.")

1246739743737 is not prime.
The 20000'th prime number is 224737.


### Practical Decorator 1: performance timing
You've seen how adding code to measure performance (execution time) is a small pain and clutters up your code.
Let's define a re-usable decorator to do the job...

In [114]:
def performance(f):
    """ Return a decorator that will execute function f and record the time it takes to execute."""
    def performance_timer(*args):
        start = time.perf_counter()
        v = f(*args)
        end = time.perf_counter()
        print(f'Performance of {f.__name__}{args}: {end-start} seconds.')
        return v
    return performance_timer

In [115]:
timed_nth_prime = performance(nth_prime)
print(f"The {Nth}'th prime number is {timed_nth_prime(Nth)}.")

Performance of nth_prime(20000,): 1.8073844569998982 seconds.
The 20000'th prime number is 224737.


### Practical Decorator 2: caching

When a computationally expensive result needs to be re-computed many times, it can be advantageous to pre-compute the values and store them in a lookup table (sound familiar?).
This strategy is called "caching" (as in, creating a "cache" of pre-defined values).
A decorator allows us to write the cache logic once and re-use it for any function...

In [116]:
def cache(f):
    """ return a version of function f that caches each result for use in future calls with the same arguments """
    result_cache = {}
    def cache_it(*args):
        if args in result_cache:
            return result_cache[args]
        else:
            result = f(*args)
            result_cache[args] = result
            return result
    return cache_it

P.S.  don't actually use this version, use built-in [functools.cache](https://docs.python.org/3/library/functools.html?highlight=functools#functools.cache) instead!

In [117]:
# Minimal example...
def f(a, b):
    print(f'f({a}, {b})')
    return a+b

cached_f = cache(f)
cached_f(1, 2)  # sum is only computed here...
cached_f(3, 4)  # and here.
cached_f(3, 4)  # rest of calls resolved by cache!
cached_f(1, 2)
cached_f(3, 4)

f(1, 2)
f(3, 4)


7

A more realistic example...

In [118]:
cached_nth_prime = performance(cache(nth_prime))
print(f"The {Nth}'th prime number is {cached_nth_prime(Nth)}.")

Performance of cache_it(20000,): 1.2340050459997656 seconds.
The 20000'th prime number is 224737.


In [119]:
print(f"The {Nth}'th prime number is {cached_nth_prime(Nth)}.")

Performance of cache_it(20000,): 1.4649995137006044e-06 seconds.
The 20000'th prime number is 224737.


## Syntactic Sugar:  the @decorator syntax
Because decorators are such a powerful way to re-use code, python provides some syntax sugar to make them even sweeter...

In [120]:
@performance
@cache
def any_function(n):
    return nth_prime(n)

print(f"The {Nth}'th prime number is {any_function(Nth)}.")
print(f"The {Nth}'th prime number is {any_function(Nth)}.")
print(f"The {Nth}'th prime number is {any_function(Nth)}.")

Performance of cache_it(20000,): 1.1970966970002337 seconds.
The 20000'th prime number is 224737.
Performance of cache_it(20000,): 1.8199998521595262e-06 seconds.
The 20000'th prime number is 224737.
Performance of cache_it(20000,): 9.880004654405639e-07 seconds.
The 20000'th prime number is 224737.


## One Final Detail
You may notice `performance` is reporting on `cache_it` - the inner function from the cache decorator :-(
This can be fixed by simply re-naming the function returned by the decorator.
Luckily Python has a nice decorator for that :-)
See [functools.wraps](https://docs.python.org/3.8/library/functools.html?highlight=functools#functools.wraps)